In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
train_transform = transforms.Compose([
    # Random augmentations (different each time!) 
    transforms .RandomHorizontalFlip(p=0.5), 
    transforms .RandomRotation(degrees=10), 
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    )
])

In [4]:
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=train_transform
)

test_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=val_transform
)

100%|██████████| 170M/170M [00:01<00:00, 104MB/s]


In [5]:
train_loader =  DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4, #Permite que dois núcleos da CPU preparem os dados simultaneamente.
    pin_memory=True
               
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4, #Permite que dois núcleos da CPU preparem os dados simultaneamente.
    pin_memory=True
)

In [6]:
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate=32, bn_size=4):
        super().__init__()
        
        # Bottleneck (Redução de dimensão)
        self.dimension_reduction = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)
        )
        
        # Feature extraction (Extração de características)
        self.feature_extraction = nn.Sequential(
            nn.BatchNorm2d(bn_size * growth_rate),
            nn.ReLU(inplace=True),
            nn.Conv2d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
        )

    def forward(self, x):
        out = self.dimension_reduction(x)
        out = self.feature_extraction(out)
        # Concatenação: a essência da DenseNet
        return torch.cat((x, out), 1)

In [7]:
class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate=32, bn_size=4):
        super().__init__()
        self.layers = nn.ModuleList()
        
        for i in range(num_layers):
            # O número de canais de entrada aumenta a cada camada pelo growth_rate
            layer = DenseLayer(
                in_channels + i * growth_rate, 
                growth_rate, 
                bn_size
            )
            self.layers.append(layer)

    def forward(self, x):
        features = x
        for dense_layer in self.layers:
            features = dense_layer(features)
        return features

In [8]:
class TransitionLayer(nn.Module):
    def __init__(self, in_channels, compression_factor=0.5):
        super().__init__()
        out_channels = int(in_channels * compression_factor)
        
        self.transition = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.transition(x)

In [9]:
class DeseNet(nn.Module):
    def __init__(self, block_config=(6, 12, 24, 16), growth_rate=32, bn_size=4, compression_factor=0.5, num_classes=10):
        super().__init__()
        
        # 1. Camada Inicial (Features iniciais)
        num_features = 64
        self.features = nn.Sequential(
            nn.Conv2d(3, num_features, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(num_features),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

       # 2. Criação dos Blocos Densos e Transições
        self.dense_blocks = nn.ModuleList()
        for i, num_layers in enumerate(block_config):
            # Adiciona o Bloco Denso
            block = DenseBlock(num_layers, num_features, growth_rate, bn_size)
            self.dense_blocks.append(block)
            num_features = num_features + num_layers * growth_rate
            # Adiciona Camada de Transição (exceto após o último bloco)
            if i != len(block_config) - 1:
                transition = TransitionLayer(num_features, compression_factor)
                self.dense_blocks.append(transition)
                num_features = int(num_features * compression_factor)

        # 3. Classificador Final
        self.classifier = nn.Sequential(
            nn.BatchNorm2d(num_features),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        for block in self.dense_blocks:
            x = block(x)
        x = self.classifier(x)
        return x

In [10]:
model = DeseNet().to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [11]:
print("Iniciando o treinamento...")
model.train()

for epoch in range(51):
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    if epoch % 10 == 0:
        print("training epoch: ", epoch)
        print("loss:", running_loss/len(train_loader))

Iniciando o treinamento...
training epoch:  0
loss: 1.434458031145203
training epoch:  10
loss: 0.5041612538763934
training epoch:  20
loss: 0.4083582736227823
training epoch:  30
loss: 0.36826033934074287
training epoch:  40
loss: 0.34944606031222114
training epoch:  50
loss: 0.3296953020117167


In [12]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        images = image.to(device)
        labels = label.to(device)
        output = model(images)
        _, predicted = output.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy:", 100 * correct / total)

Accuracy: 87.13
